In [5]:
import getopt
import logging
import os
import sys
# import traceback
import tweepy
import pandas as pd 
import numpy as np
import re
#import skimage.io as io  

In [114]:
# Generate your own at https://apps.twitter.com/app
CONSUMER_KEY = 'q8svcQ1GKW2yknY8MCZLvcO7w'
CONSUMER_SECRET = 'kk9eMhfIMVxoDEoKR63ddWooW87Ya7IgUt5oC31S0TpAXeiMdh'
OAUTH_TOKEN = '917762487608659970-G1v4Nr01JQA9UKqO1HP4g4bPwKT7LAr'
OAUTH_TOKEN_SECRET = 'p1Zp4ophwRbRvR5yET3ppXWWg7fEshIyWwby9vTBxR9CF'

# connect to twitter
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
api = tweepy.API(auth)

# batch size depends on Twitter limit, 100 at this time
batch_size = 100

#Some emojis have character length of more than 1
emoji_threshold = 3  

In [115]:
file = 'cleaned_img_train_plaintext.txt'
#file = 'img_train_plaintext.txt'
data = pd.read_csv(file, sep='\t', encoding = 'utf8', engine='c', header = 0)

#data = pd.DataFrame([j for i,j in enumerate(data.values) if len(data.iloc[i, 2].split(',')) == 1], columns=['id','imgid','annotations'])

data.head()

,id,imgid,annotations
0,742240402288345088,http://pbs.twimg.com/media/CkzuE33WsAA2cUB.jpg,229
1,742427564535353349,http://pbs.twimg.com/media/Ck2h7lzXIAA0Aqw.jpg,658
2,747199027649871872,http://pbs.twimg.com/media/Cl6UJF1XIAADDAJ.jpg,1466
3,743426787267338240,http://pbs.twimg.com/media/ClEupvHUYAA0lPF.jpg,1381
4,745081041631612928,http://pbs.twimg.com/media/ClaNsdLWkAAHti4.jpg,1380


In [116]:
def locate_emoji(emoji_pattern, text: str):
    emoji = ''.join(emoji_pattern.findall(text))
    try:
        index = text.index(emoji)
    except:
        index = -emoji_threshold
    return emoji, index

In [130]:
def get_tweets(twapi, data):
    '''
    Fetches content for tweet IDs in a file using bulk request method,
    which vastly reduces number of HTTPS requests compared to above;
    however, it does not warn about IDs that yield no tweet.
    `twapi`: Initialized, authorized API object from Tweepy
    '''
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u'\U00010000-\U0010ffff'
        u"\u200d"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\u3030"
        u"\ufe0f"
        "]+",
        flags=re.UNICODE)

    tweet_ids = data.id.values.tolist()
    emoji_labels = data.annotations.values.tolist()

    all_tweets = []
    labels = []
    i = 0  #for debug
    # process list of ids until it's empty
    while len(tweet_ids) > 0:
        if len(tweet_ids) < batch_size:
            tweets = twapi.statuses_lookup(
                id_=tweet_ids, include_entities=False, trim_user=True)
            tweet_ids = []
        else:
            tweets = twapi.statuses_lookup(
                id_=tweet_ids[:batch_size],
                include_entities=False,
                trim_user=True)
            tweet_ids = tweet_ids[batch_size:]

        for tweet in tweets:

            #removes the link of the tweet
            text = re.sub(r'http\S+', '', tweet.text).strip(' ')
            
            #Remove tweets where emoji is not at the end
            emoji, index = locate_emoji(emoji_pattern, text)
            
            if index >= len(text) - emoji_threshold:
                #removes the emojis from the text
                text = emoji_pattern.sub(r'', text).strip(' ')

                #then appends the tweet and emoji to our final dataset
                all_tweets.append(np.array([text]))
                labels.append(emoji)

        i += 1
        if i == 20:
            break
            
            
    features = np.array(all_tweets)
    labels = np.array(labels)
    return features, labels

In [131]:
np.savetxt('tweets.txt')
np.savetxt('emojis.txt')

array([['Kicking it at @arteryrcdings'],
       ['Go England'],
       ["RT @el_kdoe: I'm single"],
       ['RT @LilJDuggy: My boy Buddy all grown up now'],
       ['RT @EliasTheodorou: The one and only, @TheBigPygmy appearing on @ebaumsworld today. A man doing great things inside the cage &amp; out.'],
       ['Not just for ladies'],
       ['RT @enews: Laila Ali takes the #BETAwards stage to pay tribute to her late father, Muhammad Ali.'],
       ['I try anything'],
       ['It came in, and another dollar is in @Oceana hands to save lives that are so critical to the balance of our world.'],
       ['RT @Gh15_Yolanda: Summer love'],
       ['San Francisco sunsets'],
       ['I will miss chasing the sunsets here #OnlyOnGuam'],
       ['love my friends'],
       ['RT @courtneyadele_: #Tbt New York City Loss Photos'],
       ['What the duck!'],
       ['RT @oridaganjazz: Backstage @120Diner getting ready to introduce the amazing @gabi_epstein as part of the @TorontoJazzFest! #TDmusic'],


In [132]:
y

array(['✌️', '💪', '😒', '💔', '🙏', '👌', '❤️', '😋', '🌍', '💙', '👌', '🌅', '💕',
       '🌚', '😹', '🎶', '😘', '❤️', '💖', '🎥', '💟', '💖', '😊', '😁', '🌸', '🐐',
       '🏉', '😢', '😏', '💕', '😱', '😛', '🙄', '😱', '♠️', '💀', '😍', '😕', '😁',
       '💜', '💛', '😌', '🔥', '🐴', '😍', '❤️', '😂', '💚', '🍸', '❤️', '💕', '❤️',
       '👍', '😂', '😜', '😍', '🍐', '👍', '💖', '❤️', '😍', '😛', '😉', '😕', '🇨🇩',
       '❗️', '😊', '😄', '❤️', '😂', '🎥', '✨', '😉', '😂', '💣', '❤️', '🎉', '👉',
       '😊', '😒', '🏆', '👋', '😍', '☺️', '💕', '😍', '🇺🇸', '❤️', '🤐', '😹', '😍',
       '💙', '😊', '😁', '✨', '💙', '😭', '🙄', '👽', '💕', '😜', '👑', '😍', '🌴',
       '😃', '😭', '🔊', '😝', '😅', '😇', '❤️', '💕', '😂', '💓', '👉', '😍', '💋',
       '🐾', '🐸', '😸', '😨', '💭', '💕', '💗', '😜', '💘', '💓', '😘', '😊', '🙋',
       '🍎', '😞', '😄', '😕', '😂', '😂', '💞', '🐥', '🌸', '⚽️', '🔥', '😍', '✊',
       '😂', '💯', '😘', '💞', '😈', '😊', '😩', '😊', '☎️', '😰', '🙏', '☺️', '💗',
       '👀', '🌚', '❤️', '🐻', '👍', '👀', '😂', '😂', '😍', '💭', '🇵🇱', '🔍', '😂',
       '😋', '😎', '😍', '😅', '🌟', '😜', '💯', '